<a href="https://colab.research.google.com/github/stefanoBam/Admission_Algorithm/blob/main/Admission_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull Files from Github

In [1]:
!git clone https://github.com/stefanoBam/Admission_Algorithm.git

#!git clone https://github.com/facebookresearch/llama.git

Cloning into 'Admission_Algorithm'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 210 (delta 63), reused 158 (delta 30), pack-reused 0
Receiving objects: 100% (210/210), 159.10 KiB | 4.30 MiB/s, done.
Resolving deltas: 100% (63/63), done.


## Fuzzy search

In [2]:
!pip install thefuzz
!pip install python-Levenshtein
!pip install python-Levenshtein-wheels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein-wheels: filename=python_Levenshtein_wheels-0.13.2-cp310-cp310-linux_x86_64.whl size=114852 sha256=f7a2c8591b0a63b22169751baf80c4972e5d315df396a874152858f4dee1b7cd
  Stored in directory: /root/.cache/pip/wheels/1e/5e/81/35710bb7ed33fa74f4a0980836eda411dd966f38f237992dc4
Successfully built python-Levenshtein-wheels


## LLM

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00


# Functions

In [ ]:
## Import and intitialize LLM
## WARNING takes a while to download a 750Gb model
#from transformers import AutoModel
#model = AutoModel.from_pretrained("UFNLP/gatortron-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/713M [00:00<?, ?B/s]

In [3]:
## Import libraries and open CSV

def initialize_csv(csv_name):
  #print(os.path.isfile(csv_name))
  df_0 = pd.read_csv(csv_name)
  df = pd.DataFrame(df_0)
  #print(df.to_string())

  col1 = "Diagnosis/Presenting problem"
  col2 = "Specific Factors 1"
  col3 = "Specific Factors 2"
  col4 = "Admitting Service"
  col5 = "Notes"
  col6 = "System"
  col7 = "Mechanism"
  #print(df.columns)
  return(df, col1, col2, col3, col4, col5, col6, col7)

In [24]:
## Parse User Input
def parse_user(prompt, choices = 0):
  if prompt: print("\n"+prompt)
  if choices: print(choices)
  input1 = str(input())
  #TODO: Add parsing if necessary
  return input1

## LLM zero-shot classifier implementation with successful passing of label list, and extraction of best scoring classification

In [ ]:
# Imports and sets up a zero-shot classifier
from transformers import pipeline
#classifier = pipeline('zero-shot-classification',model="UFNLP/gatortron-base")
classifier = pipeline('zero-shot-classification')

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#Calls initialize_csv function and assigns the output values
df, col1, col2, col3, col4, col5 = initialize_csv('Admission_Algorithm/Admit_table_csv.csv')
# Converts col1 of the dataframe to a list, then converts to set to remove duplicates, then converts back to list to pass to classifier
labels = list(set(df[col1].tolist()))
output = classifier("broken rib", candidate_labels = labels)


In [ ]:
max_score = max(output['scores'])
max_index = output['scores'].index(max_score)
print(output['sequence'] + ": " + output['labels'][max_index])

broken rib: Rhabdomyolysis


In [ ]:
#function that takes the output from the LLM (i.e., presenting problem), the cols from the csv, and outputs the final admitting service
def find_admitting_service(presenting_prob, col1, col2, col3, col4, col5):
  indeces = []
  specific_factor_1 = set()
  specific_factor_2 = set()
  for i in range(len(col1)): #finding all the rows that have the same presenting problem
    if presenting_prob == col1[i]: #this should probably be better implemented (if the input wont always string match exactly to what's in the csv)
      indeces.append(i)

  for i in indeces: #adding the specific factors in a set (if there are any)
    if col2[i] != "":
      specific_factor_1.add(col2[i])
    if col3[i] != "":
      specific_factor_2.add(col3[i])

  if not bool(specific_factor_1): #if the set is not empty, have the user choose a specific factor
    prompt1 = "please choose from the following choices:" + str(specfic_factor_1)
    choice_1 = parse_user(prompt1)

  #next steps: have a counter to count how many times specific factor 1 shows up (bc if there are multiple counts, then user will have to choose from specific factor 2), if not, then we can output final admiting service

## Fuzzy Search

In [17]:
from thefuzz import fuzz, process
import csv
import pandas as pd
import numpy as np
import os
import json


In [6]:
#function that will take the user input,df,specified row to search in to match userInput, and outputs a new df with top matches (highest ratio from fuzzy search)
def fuzzysearch(userInput, df, r, top1 = False):

  #compute a ratio between user input and "text" column of df (block of code that I copy pasted from below)
  c = []
  for index, row in df.iterrows():
      b = ''.join(str(x) for x in row[r])
      c.append ([index,fuzz.token_set_ratio(userInput,b),b])

  cT = np.array(c).T
  #df['ratio'] = list(map(int, cT[1]))
  df.loc[:, 'ratio'] = list(map(int, cT[1])) #changed the above line to this line due to warning messages

  max_ratio = df["ratio"].max() #finds the best match

  #sub1.iloc[0:0]
  if top1: #if top1 is True (specified in the parameters)
    sub1 = df.loc[df['ratio'] == max_ratio]
  else: #default: if top1 is not specified in the parameters as True, the default will return a table w max ratio-5
    sub1 = df.loc[df['ratio'] >= (max_ratio - 5)] #subtable with the best ratios (anything that's >= max-5)

  return sub1

In [71]:
#function that will take the user input,df,specified row to search in to match userInput, and outputs a new df with top matches (highest ratio from fuzzy search)
def explicitSearch(userInput, df, r):

  """
  #compute a ratio between user input and "text" column of df (block of code that I copy pasted from below)
  c = []
  for index, row in df.iterrows():
      b = row[r]
      if userInput in b:
        c.append ([index,b])
  """

  sub1 = df.loc[(userInput in df[r]) or df[r].isnull()]
  sub1.to_csv("sub1_content.csv")
  return sub1

current issues:

*   not too sure if fuzzy matching/the scoring system works (i.e., if write "bone fracture" the output for prompt2 (i.e., the closest diagnosis/presenting problem) don't make sense)
*   the specific factor 1 and 2 isn't fully functional yet...


In [61]:
import math
df, col1, col2, col3, col4, col5, col6, col7 = initialize_csv('Admission_Algorithm/Admit_table_wCat_csv.csv')


#Compresses the text contents from all the columns of the dataframe into a list with one column, comma seperated text
##This will be what we fuzzy search into
df['text'] = df[[col1, col2, col3, col4, col5	]].astype(str).values.tolist()

# Converts col1 of the dataframe to a list, then converts to set to remove duplicates, then converts back to list to pass to classifier
labels = list(set(df[col1].tolist()))

# Import initial button categories from categories.json
cat_file = open("Admission_Algorithm/categories.json")
categories = json.load(cat_file)
system_labels = categories["system"]
mechanism_labels = categories["mechanism"]
n = 0
print(df[col7][n])
print(df[col7].isnull().iloc[n])
##print(test.isnull().all())


nan
True


In [73]:
#Incoporates code from Robert et. Al <3
#Calls initialize_csv function and assigns the output values
df, col1, col2, col3, col4, col5, col6, col7 = initialize_csv('Admission_Algorithm/Admit_table_wCat_csv.csv')


#Compresses the text contents from all the columns of the dataframe into a list with one column, comma seperated text
##This will be what we fuzzy search into
df['text'] = df[[col1, col2, col3, col4, col5	]].astype(str).values.tolist()

# Converts col1 of the dataframe to a list, then converts to set to remove duplicates, then converts back to list to pass to classifier
labels = list(set(df[col1].tolist()))

# Import initial button categories from categories.json
cat_file = open("Admission_Algorithm/categories.json")
categories = json.load(cat_file)
system_labels = categories["system"]
mechanism_labels = categories["mechanism"]

system_selection = parse_user("Please select from the following systems:\n" + str(system_labels))
mechanism_selection = parse_user("Please select from the following mechanisms:\n" + str(mechanism_labels))


sub0_sys = explicitSearch(system_selection, df, col6)
sub0_mech = explicitSearch(mechanism_selection, sub0_sys, col7)


prompt1 = parse_user("What is the presenting problem? Please use the shortest descriptor possible.")
ratios = []

# I copy pasted this next block of code (that's been commented out) into a function that is defined above (i.e., fuzzysearch())
"""
#compute a ratio between user input and "text" column of df
c = []
for index, row in df.iterrows():
    b = ''.join(str(x) for x in row['text'])
    c.append ([index,fuzz.token_set_ratio(prompt1,b),b])

cT = np.array(c).T
df['ratio'] = list(map(int, cT[1]))

max_ratio = df["ratio"].max() #finds the best match

#sub1.iloc[0:0]
sub1 = df.loc[df['ratio'] >= (max_ratio - 5)] #subtable with the best ratios (anything that's >= max-5)
"""
sub1 = fuzzysearch(prompt1,sub0_mech,"text")

#asking the user to choose best match
prompt2 = parse_user("Which of the following categories is most likely?", list(set(sub1[col1].tolist())))

sub2 = fuzzysearch(prompt2,sub1,col1,True)

#same thing, this next block of code seems very similar as the first block that's been commented out so i just call fuzzysearch() instead, but added a "top1" parameter to differentiate between the max_ratio and max_ratio - 5
"""
##TODO: functionalize below?
q = []
for index, row in sub1.iterrows():
    b = ''.join(str(x) for x in row[col1])
    q.append ([index,fuzz.token_set_ratio(prompt2,b),b])

qT = np.array(q).T


max_ratio = sub1["ratio2"].max()
sub2 = sub1.loc[sub1['ratio2'] == max_ratio]
"""
if sub2.shape[0] == 1:
  print("Admitting service: " + str(sub2[col4].values[:]))

##ADD another layer of search with the first subset column
#this is not done yet
else:
  sf1 = sub2[col2]
  sf2 = sub2[col3]
  print(sf1)
  print(sf2)

  if not sf1.isnull().all(): #check if specifying factor 1 is empty or not, if it is, goes on to see if there are any specifying factor 2
    prompt_sf1 = parse_user("Please choose a specifying factor", list(set(sf1.tolist())))
    sub3 = fuzzysearch(prompt_sf1, sub2, col2)
    print(sub3)

  elif not sf2.isnull().all():
    prompt_sf2 = parse_user("Please choose a specifying factor", list(set(sf2.tolist())))
    sub3 = fuzzysearch(prompt_sf2, sub2, col3)
    print(sub3)

  else: #if no specifying factor, list all possible admitting services
    print("\nAdmitting services: " + str(sub2[col4].values[:])) #prints all the possible admitting services if there are no specific factors 1 and 2




Please select from the following systems:
['respiratory', 'HEENT', 'cardiovascular', 'gastrointestinal', 'genitourinary', 'neurological', 'psychiatric', 'skin', 'MSK']
MSK

Please select from the following mechanisms:
['vascular', 'infectious', 'neoplastic', 'iatrogenic/intoxication', 'traumatic']
traumatic

What is the presenting problem? Please use the shortest descriptor possible.


KeyboardInterrupt: Interrupted by user

# Main

In [ ]:
## Main
import csv
import pandas as pd
import numpy as np
csv_name = "Admission_Algorithm/Admit_table_csv.csv"
prompt1 = "What is the presenting problem? Please describe."
initialize_csv(csv_name)
input1 = parse_user(prompt1)

# Old Code

In [ ]:
#Old hard-coded method
indeces = []
for i,r in df[[col1]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if not str(r[0]).find(input1):
        print(r[0])
        indeces.append(i)

print(indeces)
secondrow = set([])
for i in indeces:
    secondrow.add(df[col2][i])
print("Pick one of the following options:")
print(secondrow)
input2=str(input())
#for in [i]

for i,r in df[[col2]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if i in indeces:
        if not str(r).find(input2):
            print(r)

'''
for i,r in df[[col2]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if not str(r[0]).find(input1):

        if not str(r[1]).find(input2):
            if not str(r[2]).find(input3):
                print(r[3])
'''

# Notes

In [ ]:
#open file
#get input from user
#take input from user -> parse
#call algorithm to take two inputs: user input text and csv (can then fork to call chatGPT instead of csv in future)

#Next steps:
#Find open-source Python LLM available and easy to use to search the csv
#LLama2 etc. read which is best for our use-case
  #can use google scholar to find

#ask user's perspective on asnwer, do you agree with reccomendation, and add comments
  #store back in excel fiel with query, answer, comments -> for further